In [2]:
import psi4
import numpy as np
from helper_CQED_RHF import *

To-Do:

1. Check that setting C, D, eps from CQED-RHF outputs reproduces SCF grad for lambda = 0, 0, 0
2. Try to match SCF grad from the template notebook when you set lambda = 0, 0, 0.  In principle, it should match
   the sum of the Pulay force, J and K derivs, H_0 derivs, nuclear deriv
3. Once verified, implement the dipole and quadrupole deriv terms only

In [3]:

# options for H2O
psi4_options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}

psi4.set_options(psi4_options_dict)

# electric field for H2O - polarized along z-axis with mangitude 0.05 atomic units
lambda_vector = np.array([0.0, 0.0, 0.05])

molecule_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
no_reorient
no_com
symmetry c1
"""

test_mol = psi4.geometry(molecule_string)
et, wfn_t = psi4.energy("scf", return_wfn=True)


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sat Jun 14 09:00:47 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

In [4]:
# run a psi4 rhf calculation and return the wavefunction object

# sest up molecule object
molecule = psi4.geometry(molecule_string)
nuc_d = molecule.nuclear_dipole()

# get the nuclear repulsion contribution to the gradient
nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())

# Set up Psi4 options
psi4.set_options(psi4_options_dict)

# run energy calculation and return wavefunction
rhf_e , wfn = psi4.energy('scf', return_wfn=True)

# get the rhf gradient
_expected_gradient = np.asarray(psi4.gradient('scf'))

# compute the QED-RHF energy and density matrix
cqed_dict = cqed_rhf(lambda_vector, molecule_string, psi4_options_dict)

# parse dictionary for ordinary RHF and CQED-RHF energy
_rhf_e = cqed_dict["RHF ENERGY"]
_cqed_rhf_e = cqed_dict["CQED-RHF ENERGY"]

# confirm the rhf energy from this method mmatches psi4
assert np.isclose(_rhf_e, rhf_e)

# parse dictionary for density matrix
_cqed_rhf_D = cqed_dict["CQED-RHF DENSITY MATRIX"]
_cqed_rhf_C = cqed_dict["CQED-RHF C"]
_cqed_rhf_eps = cqed_dict["CQED-RHF EPS"]
_cqed_rhf_F = cqed_dict["CQED-RHF FOCK"]






Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sat Jun 14 09:00:47 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
  

In [5]:
wfn.Ca().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_C)
wfn.Cb().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_C)
wfn.Da().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_D)
wfn.Db().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_D)
wfn.epsilon_a().nph[0][:] = psi4.core.Vector.from_array(_cqed_rhf_eps)
wfn.epsilon_b().nph[0][:] = psi4.core.Vector.from_array(_cqed_rhf_eps)

In [6]:
# this is the total rhf gradient with the RHF density, etc, stored in wfn
up_scf_gradient = psi4.core.scfgrad(wfn)


*** tstart() called on CHEM9QDFT72ALT
*** at Sat Jun 14 09:00:53 2025


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000     0.000000000000     0.000000000000    15.994914619570
         H            0.000000000000     0.757000000000     0.587000000000     1.007825032230
         H            0.000000000000    -0.757000000000     0.58

In [7]:
#### call scfgrad for electron-only part of gradient ####
print(np.asarray(up_scf_gradient))
#[[-1.47661150e-16  2.35977968e-15 -1.54615212e-02]
# [ 2.32684749e-16  1.06983093e-02  7.73076058e-03]
# [-8.50235988e-17 -1.06983093e-02  7.73076058e-03]]

[[-1.47322240e-16  2.42139624e-15 -1.56873252e-02]
 [ 2.32316861e-16  1.07313369e-02  7.84366262e-03]
 [-8.49946215e-17 -1.07313369e-02  7.84366262e-03]]


In [ ]:
print(gradient)